In [ ]:
#step1

In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [ ]:
#step2

In [ ]:
# setting mediapipe holistics
#we are creating 2 variables ,one for mediapipe holistic and second one for mediapipe drawing utilities
#mediapipe holistoc and mediapipe drawing utilites are 2 components in the mediapipe 
#mediapipe holistic helps in pose estimation  whereas the mediapipe drawing utilities helps in visualizing the results for the developers
mp_holistic =mp.solutions.holistic #imports the holistic module from the solutions which is a sub module in mediapipe
mp_drawing=mp.solutions.drawing_utils #imports the drawing module from the solutions which is a sub module in mediapipe

In [ ]:
#initially when we get the feed it is in the form of BGR(blue,green,red),detection are made using mediapipe .mediapipe works with images in
#RGB(red,green,blue) format.so we make transformations using opencv.The opencv works with the images in BGR format

In [ ]:
def mediapipe_detection(image,model):   #setting a function for detection.here model is mediapipe holistic model
    image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)  #converts image from BGR 2 RGB color space.cvtcolor() is function that changes imag
    image.flags.writeable=False   #sets image unwriteable so that the mediapipe won't the image data during processing
    results=model.process(image)  #process() is a function . It processes the image using the holistic model
    image.flags.writeable=True    #sets back image to writeable after processing
    image=cv2.cvtColor(image,cv2.COLOR_RGB2BGR)  #converts image from RGB to BGR color space
    return image,results          #return the processed image,detection results


In [ ]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [ ]:
def draw_styled_landmarks(image,results):#styling the landmarks like size,colors etc
    # Draw face connections and styling face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                             mp_drawing.DrawingSpec(color=(80,110,10),thickness=1,circle_radius=1),
                             mp_drawing.DrawingSpec(color=(80,256,121),thickness=1,circle_radius=1)
                             ) 
    # Draw pose connections  and styling pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(80,22,10),thickness=2,circle_radius=4),
                               mp_drawing.DrawingSpec(color=(80,44,121),thickness=2,circle_radius=2)
                             ) 
    # Draw left hand connections  and styling left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(121,22,76),thickness=2,circle_radius=4),
                               mp_drawing.DrawingSpec(color=(121,44,250),thickness=2,circle_radius=2)
                             ) 
    # Draw right hand connections  and styling right hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=4),
                              mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2)
                             ) 

In [ ]:
cap =cv2.VideoCapture(0)  #accesing webcam.default index of webcam is 0,storing everything got from video in cap avriable
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():                   #checking whether the webcam is open or not.so used isOpened() function 
        ret,frame=cap.read()                #reading the feed.while reading, we are getting 2 values they are return value(true/false) ,image i.e frame
        image,results=mediapipe_detection(frame,holistic)
        print(results)
        draw_styled_landmarks(image,results)        #draw landmarks
        cv2.imshow('opencv feed',image)     #showing what has read  the user."opencv feed" is name given to frame
        if cv2.waitKey(10) & 0xFF==ord('q'):#breaking the loop.  if the current key pressed is 'q' the loop breaks
            break                                                           
    cap.release()                           #releases the webcam
    cv2.destroyAllWindows()                 #closes all opencv windows

In [ ]:
draw_landmarks(frame, results)

In [ ]:
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

In [ ]:
#step3

In [ ]:
#np.array is used to create arrays
#pose_landmarks has x,y,z,visibility rest has only x,y,z
#results.pose_landmarks give the result in form of landmark{x:,y:,z:,visibility:} 
#result.pose_lanmarks.landmark give the result in form of list i.e [x: y: z: visibility:, etc]
#using flatten converts into 1-D array

In [ ]:
def extract_keypoints(results):   # function to extract key points 
    pose=np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
    lh=np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh=np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose,face,lh,rh]) #cancatenating arrays

In [ ]:
extract_keypoints(results)

In [ ]:
extract_keypoints(results).shape

In [ ]:
468*3+21*3+21*3+33*4==len(extract_keypoints(results))

In [ ]:
results.pose_landmarks

In [ ]:
results.pose_landmarks.landmark

In [ ]:
np.zeros(21*3)

In [ ]:
len(results.face_landmarks.landmark),len(results.face_landmarks.landmark)*3

In [ ]:
len(results.pose_landmarks.landmark),len(results.pose_landmarks.landmark)*4

In [ ]:
len(results.left_hand_landmarks.landmark),len(results.left_hand_landmarks.landmark)*3

In [ ]:
#step4

In [ ]:
DATA_PATH=os.path.join('MP_DATA') #DATA_PATH is path where exported data is stored i.e numpy arrays,here the exported data is extracted keypoints
actions=np.array(['hello','thanks','iloveyou']) #detecting the actions hello ,thanks,iloveyou
no_sequences=30                                 #using 30 frames do detect the action,collecting 30 videos for each action
sequence_length=30                              #each video is of length 30 frames

In [ ]:
#MP_DATA is directory
#action detection means takes sequence of data to predict the action rather than single frame

In [ ]:
#creating folders to store the data
for action in actions:  #it iterates over every action
    for sequence in range(no_sequences): #it iterates over range of numbers from o to 29(no_sequences-1)
        try:
            os.makedirs(os.path.join(DATA_PATH,action,str(sequence)))  #it is creating directory for each combination of action and sequence number 
        except:                                                        #using os.makedirs()
            pass                                                        #if directory exists it goes to except block

In [ ]:
#step 5 collecting our data

In [ ]:
cap =cv2.VideoCapture(0) 
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    for action in actions:
        for sequence in range(no_sequences):
            for frame_num in range(sequence_length):
                    ret,frame=cap.read()     #each frame is read from video feed          
                    image,results=mediapipe_detection(frame,holistic) #pose detection is performed on the image using the func and return the landmarks
                    print(results)   #on the image and pose detection results
                    draw_styled_landmarks(image,results)   #landmarks are styled on each image    
                    cv2.imshow('opencv feed',image)        #displaying the image
                    if cv2.waitKey(10) & 0xFF==ord('q'):
                        break    
                    if frame_num == 0: 
                             #message displayed started data collection
                             cv2.putText(image, 'STARTING COLLECTION', (120,200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                             cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                             # Show to screen
                             cv2.imshow('OpenCV Feed', image)
                             cv2.waitKey(2000)
                    else:
                  
                             #displays the ongoing data collection
                             cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                             # Show to screen
                             cv2.imshow('OpenCV Feed', image)
                    
                    keypoints = extract_keypoints(results)
                    npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                    np.save(npy_path, keypoints) #keypoints extraced from pose detection are saved in numpy arrays and stored in corresponding directory 
                                                #of action,sequence number,frame
    cap.release()        #releases video object                   
    cv2.destroyAllWindows()      #open cv windows are destroyed           

In [ ]:
#collection break: having breaks between each sequence collection allows you to reset and reposition yourself to collect the action from start to finish


In [ ]:
#step 6 preprocess data create labels and features

In [ ]:
from sklearn.model_selection import train_test_split # train_test_split : splits the dataset into training and testing
from tensorflow.keras.utils import to_categorical # to_categorical : conerts integer into categorical data

In [ ]:
#creating a label array that represents actions
label_map={label:num for num,label in enumerate(actions)}

In [ ]:
label_map

In [ ]:
sequences,labels=[] ,[] #sequences represents x-data ,labels represents y-data
for action in actions:
    for sequence in range(no_sequences):
        window=[]
        for frame_num in range(sequence_length):
            res=np.load(os.path.join(DATA_PATH,action,str(sequence),"{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [ ]:
sequences

In [ ]:
labels

In [ ]:
#preprocesing
X=np.array(sequences)
X

In [ ]:
labels

In [ ]:
y=to_categorical(labels).astype(int) #binary format

In [ ]:
y

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.05)

In [ ]:
X_train.shape,X_test.shape ,y_train.shape,y_test.shape#training and testing tuples size

In [ ]:
#step7 training lstm

In [ ]:
#sequential helps to build neural networks layer by layer in a sequential manner.layers are added to model one by one 
#and each is connected to previous layers
from tensorflow.keras.models import Sequential
#LSTM is a rnn layer which helps in handling the sequential data
#Dense is a fully connected neural network layer.each neuron is connected to each neuron in previous layer ,helps in learning the complex patterns
from tensorflow.keras.layers import LSTM,Dense
#TensoBoard is a visualization tool for monitoring and visualizing various aspects like model training and evaluation
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
log_dir = os.path.join('Logs')#creates the directory path named logs
tb_callback = TensorBoard(log_dir=log_dir) # tb_callback is a tensorboard callback object.tensorboard writes the log files at path log_dir

In [ ]:
#Training the model
#A sequential model architecture consists of multiple LSTM layers followed by dense layers.the activation function 'relu' is
#used in LSTM and dense layers except last dense layer that uses softmax activation function.softmax is used for multi class classification
#the model is designed to take input sequence 30 with 1662 features
#return_sequence =false means final output should be returned ,true means it should return sequence of outputs not only final output 
#64,128,32 means neurons in the layer
#activation functions decide whether the neuron should be activated or not.they are applied on output of each neuron
#Relu =max(0,x)
#softmax used in output layer in multi classification models to produce probabilites of each class

In [ ]:
model=Sequential() #sequential model
model.add(LSTM(64,return_sequences=True,activation='relu',input_shape=(30,1662))) 
model.add(LSTM(128,return_sequences=True,activation='relu'))
model.add(LSTM(64,return_sequences=False,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(actions.shape[0],activation='softmax'))

In [ ]:
actions.shape[0] #output layer has 3 neurons

In [ ]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['categorical_accuracy']) #model compiling
#optimizer:updates the weigths of neural network during training,Adam is an algorithm
#loss function:it finds the difference between actual and predicted output.categorical_crossentropy is a loss function used for multiclassclassification
#metrics:used to evaluate the performance of model.metric used is categorical_accuracy.it is used in multi class classification

In [ ]:
model.fit(X_train,y_train,epochs=100,callbacks=[tb_callback])
#epochs:The number of times the entire training dataset will be passed forward and backward through the neural network during training

In [ ]:
model.summary()

In [ ]:
#prediction

In [ ]:
res=model.predict(X_test)

In [ ]:
actions[np.argmax(res[0])]

In [ ]:
actions[np.argmax(res[2])]

In [ ]:
#saving weights

In [ ]:
model.save('action.keras')

In [ ]:
del model

In [ ]:
model.load_weights('action.keras')

In [ ]:
#metrics

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [ ]:
yhat = model.predict(X_test)

In [ ]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
multilabel_confusion_matrix(ytrue, yhat)

In [ ]:
accuracy_score(ytrue, yhat)

In [ ]:
# step 11 test in real time

In [ ]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [ ]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

In [ ]:
sequence.reverse()

In [ ]:
sequence.append('def')

In [ ]:
sequence.reverse()

In [ ]:
sequence[-30:]

In [ ]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.4

cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
     
        draw_styled_landmarks(image, results)
        
        
        keypoints = extract_keypoints(results)
           #sequence.insert(0,keypoints)
          #sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

         
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()